In [27]:
import pandas as pd
pilot_result = pd.read_csv("q5_pilot_result_1.csv")
pilot_result

/tmp/ipykernel_221555/1161817355.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pilot_result = pd.read_csv("q5_pilot_result_1.csv")


,n_name,o_pageid,l_pageid,revenue
0,NaN,49158.0,NaN,NaN
1,NaN,24583.0,NaN,NaN
2,NaN,24602.0,NaN,NaN
3,NaN,24603.0,NaN,NaN
4,NaN,24607.0,NaN,NaN
...,...,...,...,...
228165078,JAPAN,NaN,103424792.0,3708.1044
228165079,JAPAN,NaN,42506566.0,33708.1290
228165080,INDIA,NaN,7804901.0,24136.6356
228165081,VIETNAM,NaN,86780431.0,73427.9700


In [38]:
pilot_result.dropna(subset = ["l_pageid"]).groupby("n_name")["l_pageid"].nunique()

n_name
CHINA                        106769
INDIA                        106554
INDONESIA                    106964
JAPAN                        107145
VIETNAM                      107167
Name: l_pageid, dtype: int64

In [39]:
pilot_result.dropna(subset = ["o_pageid"])["o_pageid"].nunique()

26840487

In [42]:
n_pages_orders_before_join = 26840487
n_pages_lineitem_before_join = {
    "CHINA                    ": 106769,
    "INDIA                    ": 106554,
    "INDONESIA                ": 106964,
    "JAPAN                    ": 107145,
    "VIETNAM                  ": 107167
}

In [40]:
paged_data = pilot_result.dropna(subset = ["l_pageid", "o_pageid"])
group_keys = paged_data["n_name"].unique()
print("n_name groups:", group_keys)
paged_data

n_name groups: ['INDONESIA                ' 'JAPAN                    '
 'INDIA                    ' 'CHINA                    '
 'VIETNAM                  ']


,n_name,o_pageid,l_pageid,revenue
101219,INDONESIA,2034914.0,8757521.0,71689.3632
235134,JAPAN,4752134.0,20450813.0,26147.9020
275723,INDIA,6309392.0,27152431.0,89993.3216
398902,INDIA,8181132.0,35206540.0,51493.4820
437814,INDIA,8444217.0,36338873.0,10931.9552
...,...,...,...,...
227341907,VIETNAM,7900388.0,33998677.0,82518.9400
227411465,CHINA,18980501.0,81681254.0,36934.0983
227429456,INDIA,20950812.0,90159880.0,16198.6440
227477133,INDONESIA,12009435.0,51682352.0,19263.2400


In [43]:
import numpy as np
l_sum = paged_data.groupby(["n_name", "l_pageid"]).agg({"revenue": "sum"})["revenue"]/n_pages_orders_before_join
group_vars_l = {}
for group_key in group_keys:
    group_data = np.concatenate([l_sum[group_key].to_numpy(), np.zeros(int(n_pages_lineitem_before_join[group_key] - len(l_sum[group_key])))])
    group_var = np.var(group_data)
    group_vars_l[group_key] = group_var
    print(group_key, group_var)
max_var = max(group_vars_l.values())
print("Max variance:", max_var)

INDONESIA                 1.6353609274587987e-08
JAPAN                     1.7742346303712457e-08
INDIA                     1.6678441271574136e-08
CHINA                     1.6717702828068845e-08
VIETNAM                   1.7503146960443705e-08
Max variance: 1.7742346303712457e-08


In [44]:
o_sum = paged_data.groupby(["n_name", "o_pageid"]).agg({"revenue": "sum"})["revenue"]/max(n_pages_lineitem_before_join.values())
o_data = np.concatenate([o_sum.to_numpy(), np.zeros(int(n_pages_orders_before_join - len(o_sum)))])
o_var = np.var(o_data)
print(o_var)

2.1340961855282765e-05


In [45]:
avg = paged_data.groupby("n_name").agg({"revenue": "sum"}) / n_pages_orders_before_join
avgs = {}
for group_key in group_keys:
    avgs[group_key] = avg.loc[group_key]["revenue"]/n_pages_lineitem_before_join[group_key]
    print(group_key, avgs[group_key])
min_avg = min(avgs.values())
print("min_avg", min_avg)

INDONESIA                 8.879750377791686e-06
JAPAN                     9.40833911981018e-06
INDIA                     8.856811704101043e-06
CHINA                     9.068283262546225e-06
VIETNAM                   9.317874645038349e-06
min_avg 8.856811704101043e-06


In [46]:
import scipy.stats
z = scipy.stats.norm.ppf(0.999)
n = z**2 * (max_var*1.1 + o_var*1.1) / ((min_avg*0.9)**2) / (0.05**2)
print("n", n)

n 1412439948.5535138


In [ ]:
avg = pilot_result["revenue"].sum() / 